In [1]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

import numpy as np
from sklearn import datasets

import core.container
from core.container import OperationContainer
from core.record_event import RecordEvent
from core.record_event import ClearEvent

from sklearn.neighbors import KNeighborsClassifier

ClearEvent()

In [2]:
@RecordEvent
def cell_2():
    iris_X, iris_y = datasets.load_iris(return_X_y=True)
    
    return OperationContainer({"iris_X": iris_X, "iris_y": iris_y})

In [3]:
iris_X, iris_y = cell_2()


container created for variable  iris_X
container created for variable  iris_y


In [4]:
@RecordEvent
def cell_3():
    np.random.seed(0)

In [5]:
cell_3()

In [6]:
@RecordEvent
def cell_4():
    indices = np.random.permutation(len(iris_X))
    iris_X_train = iris_X[indices[:-10]]
    iris_y_train = iris_y[indices[:-10]]
    iris_X_test = iris_X[indices[-10:]]
    iris_y_test = iris_y[indices[-10:]]
    
    return OperationContainer({"indices": indices, "iris_X_train": iris_X_train, "iris_y_train": iris_y_train, "iris_X_test": iris_X_test, "iris_y_test": iris_y_test})

In [7]:
indices, iris_X_train, iris_y_train, iris_X_test, iris_y_test = cell_4()


container created for variable  indices
container created for variable  iris_X_train
container created for variable  iris_y_train
container created for variable  iris_X_test
container created for variable  iris_y_test


In [8]:
@RecordEvent
def cell_5():
    knn = KNeighborsClassifier()
    knn.fit(iris_X_train, iris_y_train)

    knn.predict(iris_X_test)
    
    return OperationContainer({"knn": knn})

In [9]:
knn = cell_5()


container created for variable  knn


In [10]:
for x in core.event.operation_events:
    print(x, "\n")
    print("related data events: \n")
    for y in x.related_data_events:
        print(y, "\n")
    print("------------")

Operation with name cell_2 and id 0 started at 2022-09-19 14:04:26.093069 and ended at 2022-09-19 14:04:26.095805, 0.002736 s elapsed, 0 related data events 

related data events: 

------------
Operation with name cell_3 and id 1 started at 2022-09-19 14:04:27.132696 and ended at 2022-09-19 14:04:27.132746, 5e-05 s elapsed, 0 related data events 

related data events: 

------------
Operation with name cell_4 and id 2 started at 2022-09-19 14:04:28.069185 and ended at 2022-09-19 14:04:28.070279, 0.001094 s elapsed, 5 related data events 

related data events: 

DataContainer with base ID 5494202704 and type <class 'numpy.ndarray'> used at 2022-09-19 14:04:28.069208, previous operation event was Operation with name cell_2 and id 0 started at 2022-09-19 14:04:26.093069 and ended at 2022-09-19 14:04:26.095805, 0.002736 s elapsed, 0 related data events 

DataContainer with base ID 5494202704 and type <class 'numpy.ndarray'> used at 2022-09-19 14:04:28.070249, previous operation event was 

In [11]:
for x in core.event.data_events:
   print(x, "\n")

DataContainer with base ID 5494202704 and type <class 'numpy.ndarray'> used at 2022-09-19 14:04:28.069208, previous operation event was Operation with name cell_2 and id 0 started at 2022-09-19 14:04:26.093069 and ended at 2022-09-19 14:04:26.095805, 0.002736 s elapsed, 0 related data events 

DataContainer with base ID 5494202704 and type <class 'numpy.ndarray'> used at 2022-09-19 14:04:28.070249, previous operation event was Operation with name cell_2 and id 0 started at 2022-09-19 14:04:26.093069 and ended at 2022-09-19 14:04:26.095805, 0.002736 s elapsed, 0 related data events 

DataContainer with base ID 5494202608 and type <class 'numpy.ndarray'> used at 2022-09-19 14:04:28.070260, previous operation event was Operation with name cell_2 and id 0 started at 2022-09-19 14:04:26.093069 and ended at 2022-09-19 14:04:26.095805, 0.002736 s elapsed, 0 related data events 

DataContainer with base ID 5494202704 and type <class 'numpy.ndarray'> used at 2022-09-19 14:04:28.070271, previous

In [12]:
for x in core.event.data_containers:
    print(x, "\n")

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.

In [13]:
parent_lookup = {oe: set() for oe in core.event.operation_events}
children_lookup = {oe: set() for oe in core.event.operation_events}

In [14]:
print(core.event.operation_event_lookup)

{0: <core.event.OperationEvent object at 0x103b31910>, 1: <core.event.OperationEvent object at 0x103b16370>, 2: <core.event.OperationEvent object at 0x146eaad00>, 3: <core.event.OperationEvent object at 0x103b11250>}


In [15]:
for oe in core.event.operation_events:
    for related_data_event in core.event.operation_event_lookup[oe.exec_uuid].related_data_events:
        parent_oe = related_data_event.prev_operation_event
        
        parent_lookup[oe].add(parent_oe)
        children_lookup[parent_oe].add(oe)

In [16]:
print(parent_lookup)
print(children_lookup)

{<core.event.OperationEvent object at 0x103b31910>: set(), <core.event.OperationEvent object at 0x103b16370>: set(), <core.event.OperationEvent object at 0x146eaad00>: {<core.event.OperationEvent object at 0x103b31910>}, <core.event.OperationEvent object at 0x103b11250>: {<core.event.OperationEvent object at 0x146eaad00>}}
{<core.event.OperationEvent object at 0x103b31910>: {<core.event.OperationEvent object at 0x146eaad00>}, <core.event.OperationEvent object at 0x103b16370>: set(), <core.event.OperationEvent object at 0x146eaad00>: {<core.event.OperationEvent object at 0x103b11250>}, <core.event.OperationEvent object at 0x103b11250>: set()}


In [17]:
from core.convert_graph import ConvertGraph
graph = ConvertGraph()

In [18]:
print(graph.active_nodes)
for edge in graph.edges:
    print(edge.oe.cell_func_name, edge.duration)
    print("sources")
    for node in edge.src.nodes:
        print(node.var.name, node.var.ver, node.size)
    print("destinations")
    for node in edge.dst.nodes:
        print(node.var.name, node.var.ver, node.size)

{'iris_X': <core.graph.node.Node object at 0x1477c17c0>, 'iris_y': <core.graph.node.Node object at 0x1477c1c10>, 'indices': <core.graph.node.Node object at 0x16e8b10d0>, 'iris_X_train': <core.graph.node.Node object at 0x16e8b1160>, 'iris_y_train': <core.graph.node.Node object at 0x16e8b1490>, 'iris_X_test': <core.graph.node.Node object at 0x16e8b19d0>, 'iris_y_test': <core.graph.node.Node object at 0x16e8b1be0>, 'knn': <core.graph.node.Node object at 0x16e8b1ca0>}
cell_2 0.002736
sources
destinations
iris_X 1 4928
iris_y 1 1312
cell_3 5e-05
sources
destinations
cell_4 0.001094
sources
iris_X 1 4928
iris_y 1 1312
destinations
indices 1 1312
iris_X_train 1 4608
iris_y_train 1 1232
iris_X_test 1 448
iris_y_test 1 192
cell_5 0.005784
sources
iris_X_train 1 4608
iris_y_train 1 1232
iris_X_test 1 448
destinations
knn 1 48


In [38]:
from algorithm.optimizer_greedy import OptimizerGreedy

graph.set_migration_speed(100000)
opt = OptimizerGreedy(graph, list(graph.active_nodes.values()))
recompute_nodes = opt.select_nodes()

for node in recompute_nodes:
    print(node.var.name, node.var.ver, node.size)

current score:  0.1408
best decrease:  0.013945999999999986
best set:  5
current score:  0.12685400000000002
best decrease:  0.06288000000000002
best set:  6
current score:  0.063974
best decrease:  0.0
best set:  4
iris_y 1 1312
knn 1 48
iris_X 1 4928


In [37]:
import matplotlib.pyplot as plt
import networkx as nx

#nx.draw(opt.compute_graph, with_labels = True)
#plt.show()

print(opt.active_node_sets)
print(opt.recomputation_edges)

recompute_idx = set()
recompute_idx.add(1)

migrate_idx = set()
migrate_idx.add(5)
migrate_idx.add(7)

print(opt.compute_migration_cost(migrate_idx))
print(opt.compute_recomputation_cost(recompute_idx))
print(opt.total_cost(migrate_idx))

{1, 4, 5, 6, 7}
{0: set(), 1: {(0, 1)}, 2: set(), 3: {(2, 3)}, 4: {(1, 4)}, 5: {(4, 5)}, 6: {(5, 6)}, 7: {(6, 7)}}
0.0784
0.002736
0.081136


In [32]:
from core.graph.search import find_path

recompute_edges = find_path(graph, recompute_nodes, core.event.operation_events)
for edge in recompute_edges:
    print(edge.oe.cell_func_name)
    print(edge.oe.cell_func_code)

iris_y_train
indices
iris_X_train
iris_X_test
iris_y_test
source oes:
cell_4
--------
cell_4
cell_4
@RecordEvent
def cell_4():
    indices = np.random.permutation(len(iris_X))
    iris_X_train = iris_X[indices[:-10]]
    iris_y_train = iris_y[indices[:-10]]
    iris_X_test = iris_X[indices[-10:]]
    iris_y_test = iris_y[indices[-10:]]
    
    return OperationContainer({"indices": indices, "iris_X_train": iris_X_train, "iris_y_train": iris_y_train, "iris_X_test": iris_X_test, "iris_y_test": iris_y_test})



In [34]:
from core.io.migrate import migrate
print(recompute_nodes, recompute_edges)
recomputation_code = migrate(recompute_nodes, recompute_edges, "numpy_notebook")
for code in recomputation_code:
    print("---------------------------------------")
    print(code)

{<core.graph.node.Node object at 0x1477c1c10>, <core.graph.node.Node object at 0x16e8b1ca0>, <core.graph.node.Node object at 0x1477c17c0>} [<core.graph.edge.Edge object at 0x16e8b1fd0>]
---------------------------------------
iris_X = data_container_dict["iris_X"]
---------------------------------------
iris_y = data_container_dict["iris_y"]
---------------------------------------
def cell_4():
    indices = np.random.permutation(len(iris_X))
    iris_X_train = iris_X[indices[:-10]]
    iris_y_train = iris_y[indices[:-10]]
    iris_X_test = iris_X[indices[-10:]]
    iris_y_test = iris_y[indices[-10:]]
    
    return indices, iris_X_train, iris_y_train, iris_X_test, iris_y_test
---------------------------------------
indices, iris_X_train, iris_y_train, iris_X_test, iris_y_test = cell_4()


In [ ]:
import sys
mods = [m.__name__ for m in sys.modules.values() if m]
print(mods)